In [307]:
import sqlite3
import pandas as pd
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


df_games = pd.read_csv("originales/games.csv")
df_teams = pd.read_csv("originales/teams.csv")
df_teamstats = pd.read_csv("originales/teamstats.csv")
df_leagues = pd.read_csv("originales/leagues.csv")
df_appearances = pd.read_csv("originales/appearances.csv")
df_players = pd.read_csv("originales/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("originales/shots.csv")



""" df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/games.csv")
df_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teams.csv")
df_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teamstats.csv")
df_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/leagues.csv")
df_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/appearances.csv")
df_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/shots.csv") """

' df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/games.csv")\ndf_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teams.csv")\ndf_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teamstats.csv")\ndf_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/leagues.csv")\ndf_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/appearances.csv")\ndf_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/players.csv", encoding="ISO-8859-1")\ndf_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/shots.csv") '

In [308]:
df_teamstats_away = df_teamstats.loc[df_teamstats["location"] == "a"]
df_teamstats_home = df_teamstats.loc[df_teamstats["location"] == "h"]
df_teamstats_merged = pd.merge(
    df_teamstats_home, df_teamstats_away, on="gameID", suffixes=["_home", "_away"]
)

df_games_merged_with_teamstats = pd.merge(
    df_games, df_teamstats_merged, on="gameID", suffixes=("", "_remove")
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    df_games_merged_with_teamstats.iloc[:, 13:34], axis=1
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["homeProbability", "drawProbability", "awayProbability"], axis=1
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["teamID_home", "teamID_away"], axis=1
)

df_games_merged_with_leagues = pd.merge(
    df_games_merged_with_teamstats, df_leagues, on="leagueID"
)

df_games_merged_with_team = pd.merge(
    df_games_merged_with_leagues,
    df_teams,
    left_on=["homeTeamID"],
    right_on=["teamID"],
    how="left",
)
df_games_merged_with_team = pd.merge(
    df_games_merged_with_team,
    df_teams,
    left_on=["awayTeamID"],
    right_on=["teamID"],
    how="left",
)

df_f_game = df_games_merged_with_team.rename(
    columns={"name_x": "liga", "name_y": "home_name", "name": "away_name"}
)
df_f_game = df_f_game.drop(
    [
        "understatNotation",
        "season_home",
        "date_home",
        "location_home",
        "goals_away",
        "goals_home",
        "season_away",
        "date_away",
        "location_away",
        "location_away",
        "teamID_x",
        "teamID_y",
    ],
    axis=1,
)


# df_appearances = pd.read_csv("appearances.csv")
# df_players = pd.read_csv("players.csv", encoding="ISO-8859-1")
# df_shots = pd.read_csv("shots.csv")

df_apperance_merged_with_players = pd.merge(
    df_appearances, df_players, on="playerID"
)





In [309]:
df_leagues

,leagueID,name,understatNotation
0,1,Premier League,EPL
1,2,Serie A,Serie_A
2,3,Bundesliga,Bundesliga
3,4,La Liga,La_liga
4,5,Ligue 1,Ligue_1


In [310]:
df2_appearances = pd.read_csv("players/appearances.csv")
df2_club_games = pd.read_csv("players/club_games.csv")
df2_clubs = pd.read_csv("players/clubs.csv")
df2_competitions = pd.read_csv("players/competitions.csv")
df2_games = pd.read_csv("players/games.csv")
df2_player_val= pd.read_csv("players/player_valuations.csv")
df2_players = pd.read_csv("players/players.csv")



""" df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")
df2_club_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/club_games.csv")
df2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")
df2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")
df2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")
df2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")
df2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv") """


' df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")\ndf2_club_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/club_games.csv")\ndf2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")\ndf2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")\ndf2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")\ndf2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")\ndf2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv") '

In [311]:

top_5_ligas=df2_competitions.loc[(df2_competitions['pretty_name']=='Bundesliga') | (df2_competitions['pretty_name']=='Laliga') | (df2_competitions['pretty_name']=='Premier League') | (df2_competitions['pretty_name']=='Serie A') | (df2_competitions['pretty_name']=='Ligue 1') ]
#!pip install thefuzz
#!pip install rapidfuzz
#!pip install python-Levenshtein
df_f_game[['date','hour']]=df_f_game['date'].str.split(' ',1,expand=True)
pd.options.mode.chained_assignment = None  # default='warn'

from fuzzywuzzy import fuzz, process


from thefuzz import process, fuzz

def match_ligue(x,col,score):

  match=process.extractOne(x, col,score_cutoff=score,scorer=fuzz.token_sort_ratio)
  
  if match:
    return match[2]
  else:
    return 2

def match_teams(x,col,score):

  match=process.extractOne(x, col,score_cutoff=score,scorer=fuzz.WRatio)
  
  if match:
    return match[2]
  else:
    return 2

def match_games(x,col, score):

  match=process.extractOne(x, col,score_cutoff=score,scorer=fuzz.WRatio)
  
  if match:
    return match[2]
  else:
    return 1

#---------------------------------------------------------------------------------------------------------
# top_5_ligas["pretty_name"]      df_teams.name
top_5_ligas['liga_new_id'] = df_leagues.loc[top_5_ligas["pretty_name"].apply(lambda x: match_ligue(x,df_leagues.name,50) ).values,["leagueID"]].values
#---------------------------------------------------------------------------------------------------------

seasons=df_f_game["season"].unique()
df2_games= df2_games[df2_games['season'].isin(seasons)]
df2_games_top5= df2_games[df2_games['competition_id'].isin(top_5_ligas['competition_id'])]
df2_games_top5=pd.merge(df2_games_top5,top_5_ligas, on='competition_id')


df2_clubs_top5=df2_clubs[df2_clubs['domestic_competition_id'].isin(top_5_ligas['competition_id'])]





df2_appearances_top5= df2_appearances[df2_appearances['game_id'].isin(df2_games_top5['game_id'])]
df2_players_top5=df2_players[df2_players['current_club_id'].isin(df2_clubs_top5['club_id'])]


#-----------------------------------------------------------------------------------------
# df2_players_top5['pretty_name']      df_players.name
#df2_players_top5['new_id'] = df_players.loc[df2_players_top5.apply(lambda x: match(x,df_players,70) ).values,"name"].values
#----------------------------------------------------------------------------------------------


df2_players_val_top5=df2_player_val[df2_player_val['player_id'].isin(df2_players_top5['player_id'])]





In [312]:
df2_games_top5['date']=pd.to_datetime(df2_games_top5['date'],format='%Y/%m/%d')
df_f_game['date']=pd.to_datetime(df_f_game['date'],format='%Y/%m/%d')



In [313]:
df2_games_top5['month_year']=df2_games_top5['date'].dt.to_period('M')
df_f_game['month_year']=df_f_game['date'].dt.to_period('M')

In [314]:
df2_games_top5['match_name']= df2_games_top5['club_home_pretty_name'] +" vs " + df2_games_top5['club_away_pretty_name']
df_f_game['match_name']= df_f_game['home_name'] +" vs " + df_f_game['away_name']

df2_clubs_top5_games=df2_games_top5.columns
df_f_game_games=df_f_game.columns

print(df2_clubs_top5_games)
print(df_f_game_games)




Index(['game_id', 'competition_id', 'competition_type', 'season', 'round',
       'date', 'home_club_id', 'away_club_id', 'home_club_goals',
       'away_club_goals', 'aggregate', 'home_club_position',
       'away_club_position', 'club_home_pretty_name', 'club_away_pretty_name',
       'home_club_manager_name', 'away_club_manager_name', 'stadium',
       'attendance', 'referee', 'url_x', 'pretty_name', 'type', 'sub_type',
       'country_id', 'country_name', 'country_latitude', 'country_longitude',
       'domestic_league_code', 'name', 'confederation', 'url_y', 'liga_new_id',
       'month_year', 'match_name'],
      dtype='object')
Index(['gameID', 'leagueID', 'season', 'date', 'homeTeamID', 'awayTeamID',
       'homeGoals', 'awayGoals', 'homeGoalsHalfTime', 'awayGoalsHalfTime',
       'xGoals_home', 'shots_home', 'shotsOnTarget_home', 'deep_home',
       'ppda_home', 'fouls_home', 'corners_home', 'yellowCards_home',
       'redCards_home', 'result_home', 'xGoals_away', 'shots_away'

In [315]:
def match_games(x,y, score):
  
  season=x['season']
  liga=x['liga_new_id']
  month_year=x['month_year']
  home_club_goals=x['home_club_goals']
  away_club_goals=x['away_club_goals']
  


  y=y.loc[(y['season']==season) & (y['leagueID']==liga) & (y['month_year']==month_year) & (y['homeGoals']==home_club_goals) & (y['awayGoals']==away_club_goals)]

  match=process.extractOne(x['match_name'], y['match_name'],score_cutoff=score,scorer=fuzz.token_set_ratio)
  
  if match:
    return match[2]
  else:
    return 1


#---------------------------------------------------------------------------------------------------------

df2_games_top5['match_name_df'] = df_f_game.loc[df2_games_top5.apply(lambda x: match_games(x,df_f_game,80),axis=1 ).values,'match_name'].values
#---------------------------------------------------------------------------------------------------------

In [316]:
df2_games_top5

,game_id,competition_id,competition_type,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,...,country_latitude,country_longitude,domestic_league_code,name,confederation,url_y,liga_new_id,month_year,match_name,match_name_df
0,2475559,ES1,domestic_league,2014,38. Matchday,2015-05-23,1084,368,2,3,...,39.326068,-4.837979,ES1,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Fc Malaga vs Fc Sevilla,Malaga vs Sevilla
1,2475556,ES1,domestic_league,2014,38. Matchday,2015-05-23,418,3709,7,3,...,39.326068,-4.837979,ES1,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Real Madrid vs Fc Getafe,Real Madrid vs Getafe
2,2475555,ES1,domestic_league,2014,38. Matchday,2015-05-23,1533,993,3,0,...,39.326068,-4.837979,ES1,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Sd Eibar vs Fc Cordoba,Eibar vs Cordoba
3,2475557,ES1,domestic_league,2014,38. Matchday,2015-05-23,940,714,3,2,...,39.326068,-4.837979,ES1,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Celta Vigo vs Espanyol Barcelona,Celta Vigo vs Espanyol
4,2475558,ES1,domestic_league,2014,38. Matchday,2015-05-23,3302,1049,2,3,...,39.326068,-4.837979,ES1,laliga,europa,https://www.transfermarkt.co.uk/laliga/startse...,4,2015-05,Ud Almeria vs Fc Valencia,Almeria vs Valencia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12676,3433222,IT1,domestic_league,2020,4. Matchday,2020-10-18,416,1390,2,3,...,42.638426,12.674297,IT1,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-10,Fc Turin vs Cagliari Calcio,Bournemouth vs Aston Villa
12677,3433216,IT1,domestic_league,2020,4. Matchday,2020-10-17,4083,506,1,1,...,42.638426,12.674297,IT1,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-10,Fc Crotone vs Juventus Turin,Crotone vs Juventus
12678,3433190,IT1,domestic_league,2020,1. Matchday,2020-09-21,5,1025,2,0,...,42.638426,12.674297,IT1,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-09,Ac Mailand vs Fc Bologna,AC Milan vs Bologna
12679,3433206,IT1,domestic_league,2020,3. Matchday,2020-10-04,4171,1025,1,0,...,42.638426,12.674297,IT1,serie-a,europa,https://www.transfermarkt.co.uk/serie-a/starts...,2,2020-10,Benevento Calcio vs Fc Bologna,Benevento vs Bologna


In [323]:
df2_clubs_top5.to_csv('equipos.csv')


In [318]:
df_teams

,teamID,name
0,71,Aston Villa
1,72,Everton
2,74,Southampton
3,75,Leicester
4,76,West Bromwich Albion
...,...,...
141,182,Nancy
142,224,Amiens
143,225,Strasbourg
144,235,Nimes


In [322]:
df2_competitions.loc[df2_competitions['pretty_name']=='Bundesliga']

,competition_id,pretty_name,type,sub_type,country_id,country_name,country_latitude,country_longitude,domestic_league_code,name,confederation,url
0,L1,Bundesliga,domestic_league,first_tier,40,Germany,51.163818,10.447831,L1,bundesliga,europa,https://www.transfermarkt.co.uk/bundesliga/sta...
